# Ontology-Based Corner Case Generation and Detection

Make use of the classical generate-and-test approach to automatically generating and detecting corner cases at the ontology-level.

## Prelude

In [1]:
from tqdm import tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches 
import random
import tabulate

%matplotlib inline
mpl.rcParams['figure.figsize'] = [8, 8]

from cc_gen.variation import *
from cc_gen.generator import *
from cc_gen.root_domain import *
from cc_gen.plausibility_filters import *

def shuffled(list):
    """ create a permutation of input list """
    l = list.copy()
    random.shuffle(l)
    return l

def setup_output_folders(name, delete_existing=False):
    """ create output folders, possibly delete pre-existing ones """
    import os
    import shutil
    if delete_existing:
        shutil.rmtree(f'./{name}', ignore_errors=True)
    os.makedirs(f'./{name}/domains', exist_ok=True)
    os.makedirs(f'./{name}/scenes', exist_ok=True)

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


## The Domain Model

Define characterization concepts for this experiment.

In [2]:
def create_characterizations(ontology):
    with ontology:
        class Moving(ontology.Entity):
            pass

        # entity is moving it has a non-zero velocity
        Imp('moving_rule').set_as_rule("""
            has_velocity(?e, ?v), greaterThan(?v, 0) -> Moving(?e)
        """)

        
        class OnTheLeft(ontology.Entity):
            pass
        
        # swrl rule:
        # entity is on the left (of the ego-car) if its 
        # lateral distance is less than that of the ego-car 
        Imp('on_the_left_rule').set_as_rule("""
            EgoCar(?x),
            has_lateral_distance(?x, ?ego_lat),
            
            Entity(?e),
            has_lateral_distance(?e, ?l),
            
            lessThan(?l, ?ego_lat) -> OnTheLeft(?e)
        """)
        
        
        class OnTheRight(ontology.Entity):
            pass
        
        # swrl rule:
        # entity is on the right (of the ego-car) if its
        # lateral distance is greater than that of the ego-car
        Imp('on_the_right_rule').set_as_rule("""
            EgoCar(?x),
            has_lateral_distance(?x, ?ego_lat),
            
            Entity(?e),
            has_lateral_distance(?e, ?l),
            
            greaterThan(?l,?ego_lat) -> OnTheRight(?e)
        """)

        
        class Crossing(ontology.Entity):
            equivalent_to = [(ontology.OnTheLeft  & 
                              ontology.has_direction.some(OneOf([ontology.north_east, 
                                                                 ontology.east, 
                                                                 ontology.south_east]))) |
                             (ontology.OnTheRight & 
                              ontology.has_direction.some(OneOf([ontology.south_west, 
                                                                 ontology.west, 
                                                                 ontology.north_west])))]
        
        
        class AtRelevantLocation(ontology.Entity):
            pass

        # swrl rule:
        # entity is at a relevant location if its within an 
        # 1.5 eog-car's length radius
        Imp('at_relevant_location_long_rule').set_as_rule("""            
            EgoCar(?x),
            has_length(?x, ?l),
            multiply(?l15, ?l, 1.5),
            
            has_euclidean_distance(?e, ?d),
            greaterThan(?d, 0),
            
            lessThanOrEqual(?d, ?l15) -> AtRelevantLocation(?e)            
        """)

        
        class Occluded(ontology.Entity):
            equivalent_to = [ontology.has_reduced_height.min(1)]
            pass
        

        class MostlyOccluded(ontology.Occluded):
            pass

        # swrl rule:
        # entity is mostly occluded if less than 13%  
        # but a positive rest of its height remains visible 
        Imp('mostly_occluded_rule').set_as_rule("""
            Occluded(?e),
            has_reduced_height(?e, ?v), 
            has_height(?e, ?h),
            multiply(?f, ?h, 0.13),
            lessThanOrEqual(?v, ?f),
            greaterThan(?v, 0) -> MostlyOccluded(?e)
        """)

        class CompletelyOccluded(ontology.Occluded):
            pass

        # swrl rule:
        # entity is completely occluded if its visible
        # height is zero or negative
        Imp('completely_occluded_rule').set_as_rule("""
            Occluded(?e),
            has_reduced_height(?e, ?v), 
            lessThanOrEqual(?v, 0) -> CompletelyOccluded(?e)
        """)

### Define Corner Case
Define corner cases to be situations in which pedestrians are occluded and moving within the relevant location

In [3]:
def create_app_domain(ontology):
    create_root_domain(ontology)
    create_characterizations(ontology)
    with ontology:        
        class CornerCase(ontology.Pedestrian):
            equivalent_to = [      
                ontology.Pedestrian &
                ontology.AtRelevantLocation & 
                (ontology.Crossing | ontology.Occluded)
            ]

## Define Parameter Space/Variation Dimensions

In this experiment the attribute ranges are given as follows

In [4]:
overview_table = []

distances_lat = [round(.5 + .2*i, 1) for i in range(21)]
overview_table.append(["range of lateral distances", f"{min(distances_lat)} to {max(distances_lat)}"])

distances_long = [round(.5 + .4*i, 1) for i in range(21)]
overview_table.append(["range of longitudinal distances", f"{min(distances_long)} to {max(distances_long)}"])


ortho_dirs = [Direction.North, Direction.West, Direction.South, Direction.East]
overview_table.append(["range of directions", f"{ortho_dirs}"])


car_speeds = [round(5 * i, 1) for i in range(9)]
overview_table.append(["range of car speed", f"{car_speeds}"])

car_widths = [1.8]
overview_table.append(["range of car width", f"{car_widths}"])

car_lengths = [4.5]
overview_table.append(["range of car length", f"{car_lengths}"])

car_heights = [1.6] 
overview_table.append(["range of car height", f"{car_heights}"])


ped_speeds = [i for i in range(15)]
overview_table.append(["range of pedestrian speed", f"{ped_speeds}"])

ped_widths = [0.5]
overview_table.append(["range of pedestrian width", f"{ped_widths}"])

ped_lengths = [0.3]
overview_table.append(["range of pedestrian length", f"{ped_lengths}"])

ped_heights = [1.7] 
overview_table.append(["range of pedestrian height", f"{ped_heights}"])

tabulate.tabulate(overview_table, tablefmt='html') 

range of lateral distances,0.5 to 4.5
range of longitudinal distances,0.5 to 8.5
range of directions,"['north', 'west', 'south', 'east']"
range of car speed,"[0, 5, 10, 15, 20, 25, 30, 35, 40]"
range of car width,[1.8]
range of car length,[4.5]
range of car height,[1.6]
range of pedestrian speed,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]"
range of pedestrian width,[0.5]
range of pedestrian length,[0.3]
range of pedestrian height,[1.7]


Using the ranges as defined above, the parameter space is given as follows:

In [5]:
variations = [
    *[EntityVariation(
        kind=Kind.Pedestrian,
        name=f'ped_{i}',
        schema=VariationSchema(
            velocity=ped_speeds,
            orientation=shuffled(ortho_dirs),
            width=ped_widths,
            length=ped_lengths,
            height=shuffled(ped_heights),
            distance_lat=shuffled(distances_lat),
            distance_long=shuffled(distances_long)
        )
    ) for i in range(4)],

    *[EntityVariation(
        kind=Kind.Vehicle,
        name=f'car_{i}',
        schema=VariationSchema(
            velocity=[0],
            orientation=[Direction.North],
            width=car_widths,
            length=car_lengths,
            height=shuffled(car_heights),
            distance_lat=[3.5],
            distance_long=shuffled([(i-1) * 6 + 3])
        )
    ) for i in range(3)],
    
    # ego car
    EntityVariation(
        kind=Kind.Ego,
        name='ego',
        schema=VariationSchema(
            velocity=shuffled(car_speeds[1:]),
            orientation=[Direction.North],
            width=car_widths,
            length=car_lengths,
            height=shuffled(car_heights),
            distance_lat=[0],
            distance_long=[0]
        )
    )
]

Specify the variation space. Make sure that there is only one ego vehicle, that no individuals overlap, and that movement of cars and pedestrians is plausible.

In [6]:
 variation_dimensions = VariationDimensions(filters=[no_overlap, left_hand_contra_car], variations=variations)

### Generate Scenes

Note that to save some time, a sample of 500 scenes are drawn. To sample some scenes, set the parameter `max_tries` for the generator.

In [7]:
BASE_IRI = "https://www.qualityminds.de/ontologies/2020/kiab/ap22/cc-gen"

name = 'experiment-1'
setup_output_folders(name, delete_existing=True)

generator = SceneGenerator(variation_dimensions, domain_factory=create_app_domain, base_iri=BASE_IRI)
r = 0
c = 0

for result in tqdm(generator):
    if result:
        onto, scene = result
        r += 1
        if onto.CornerCase.instances():
            c += 1
            generator.save_as_xml(f'{name}/domains/rdf-{r:04}.rdf.xml', onto)
            generator.save_as_png(f'{name}/scenes/scene-{r:04}.png', scene, onto)
            
f = round(c / r * 100, 1)
print(f"found {c} corner cases in {r} combinations ({f}%)")

726it [01:48,  6.71it/s]

found 47 corner cases in 49 combinations (95.9%)
